In [1]:
from scrapy import Selector
import requests
from tqdm import tqdm

In [2]:
full_url = 'http://www.edu.xunta.gal/centros/iesisaacdiazpardo/'
base_url = 'http://www.edu.xunta.gal'

output_dir = 'output'

In [3]:
def output_file(path, content):
    import os
    import errno
    
    filename = output_dir + '/' + path

    if not os.path.exists(os.path.dirname(filename)):
        try:
            os.makedirs(os.path.dirname(filename))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

    with open(filename, "w") as f:
        f.write(content)


In [4]:
def get_html(url):
    return requests.get(full_url).text

In [5]:
def get_sources_links(html):
    
    links = []
    
    css_links = Selector(text=html).xpath('//link/@href').getall()
    js_links = Selector(text=html).xpath('//script/@src').getall()

    links.extend(css_links)
    links.extend(js_links)

    return links

In [6]:
def save_html(html):
    output_file('./index.html', html)

In [7]:
def parse_links(links): 
    parsed_links = []
    
    for link in links:
        if (link[:7] != 'http://'):
            parsed_links.append(base_url + link)
        
        else:
            parsed_links.append(link)

    return parsed_links

In [8]:
def save_sources(source_links):
    for link in tqdm(source_links):
        
        source = requests.get(link).text
        
        _, path = link.split(full_url)
                
        output_file(f'./{path}', source)

In [9]:
html = get_html(full_url)
save_html(html)

unparsed_links = get_sources_links(html)
links = parse_links(unparsed_links)

save_sources(links)

100%|██████████| 50/50 [00:03<00:00, 14.98it/s]
